```

| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 하나 
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
                            .
                            .
                            .
                            .
                            .
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 백 

```

전체 구간에 대해 유전 알고리즘을 적용시킴니다.

In [1]:
# import 및 변수정의 구간
import random
import math
import re
import operator 
from random import randint

# 리스트 크기
listN = 8

# 염색체 해집단 수
N = 30

# 역 구간
Station_section = 11

# 하루에 운행하는 열차 수
One_day_train = 100

#교차율
Pc = 0.7

#변이율 역수 관계
Pm = 10

#마지막 세대수 
largest_generation = 500

#보존할 상위값
save_value = 7


In [2]:
# 01010로 이루어진 리스트를 10진수로 디코딩하는 함수

def decoding(X, Y=0):
    """
    input   : 0100로 이루어진 리스트
    return : 10진수 정수
    """
    for i, x in enumerate(X[1:]):
        Y += x* math.pow(2, listN -i -2)
    #처음 값이 0이면 음수로 return
    if X[0] == 0: return -Y
    return Y

In [3]:
#파일을 2차원 배열로 바꿔주기

def Readtxt(fname):
    """
    input  : 파일들
    return : 2차원 배열 
    """
    w = []
    with open(fname, 'r') as f :
        for line in f: #for each line in the file,
            w.append(line.split('\t'))
    return w

In [4]:
# 바뀌야됨

def ScoreFunction(X):
    """
    X에 속도 리스트를 넣어서 평가
    """
    AirportNum = Readtxt('data/Number-of-train-people.txt')
    AirportTime = Readtxt('data/Time-of-airport-train.txt')
    AtherTime = Readtxt('data/Time-of-ather-train.txt')
    sp = decoding(X)
    return (777 - (6 * sp)) * 8

In [5]:
#초기값
speedlist = [[[[random.choice([0,1]) for i in range(listN)]for j in range(Station_section)]for k in range(One_day_train)]for z in range(N)]



In [6]:
print(len(speedlist), len(speedlist[0]), len(speedlist[0][0]),len(speedlist[0][0][0]))

30 100 11 8


In [6]:
#하루 열차의 총합 구하기
def Make_Score_per_all(speedlist):
    #열차마다 점수                             #30개의 경쟁 100개의 열차
    Score_per_train = [[0 for i in range(One_day_train)]for j in range(N)]

    # 30개의 경쟁
    for k in range(N):
        # 100대의 열차
        for i in range(One_day_train):
            # 11개의 역
            for j in range(Station_section):
                #각 열차에 대해      += 11개의 구간의 점수 합                   ([30경쟁][100대열차] +=  [30경쟁][100대열차][11개의 역])
                Score_per_train[k][i] += ScoreFunction(speedlist[k][i][j])


    # 하루마다의 점수 
    Score_per_all = [0 for i in range(N)]

    # 30개의 경쟁            
    for i in range(N):
        # 100대의 열차
        for j in range(One_day_train):
             Score_per_all[i] += Score_per_train[i][j]
                
    return Score_per_all

In [63]:
Score_per_all = Make_Score_per_all(speedlist)
Score_per_all

[6890016.0,
 6826848.0,
 6682800.0,
 6813072.0,
 6787728.0,
 6849072.0,
 6825408.0,
 6931632.0,
 7021968.0,
 6857040.0,
 6862848.0,
 7103472.0,
 6861936.0,
 6753888.0,
 6791184.0,
 6667008.0,
 6824592.0,
 6766320.0,
 6908448.0,
 7098576.0,
 6893904.0,
 6915264.0,
 6928800.0,
 6690384.0,
 6906192.0,
 6930576.0,
 6918672.0,
 6946272.0,
 6873024.0,
 6857808.0]

In [64]:
print(len(speedlist), len(speedlist[0]), len(speedlist[0][0]), len(speedlist[0][0][0]),"총개수 : " ,len(speedlist) * len(speedlist[0]) * len(speedlist[0][0]) * len(speedlist[0][0][0])
)

print("-"*100)

#하루마다의 점수 (이걸로 경쟁)
print(Score_per_all[1])
#열차마다의 점수
#print(Score_per_train[1][2])
#구간마다의 점수
print(ScoreFunction(speedlist[1][2][1]))


30 100 11 8 총개수 :  264000
----------------------------------------------------------------------------------------------------
6826848.0
6120.0


In [7]:
def Score_assessment_group(Score_per_all):
    # 적합도 비율 찾기
    speedlist_evaluation = [[Score_per_all[i], 0, i] for i in range(N)]

    Sum_of_fitness = 0
    for i in range(N): 
        Sum_of_fitness += Score_per_all[i]
    for i in range(N):
        if i == 0:
            speedlist_evaluation[i][1] = speedlist_evaluation[i][0]/Sum_of_fitness
        else:
            speedlist_evaluation[i][1] = speedlist_evaluation[i-1][1]  + speedlist_evaluation[i][0]/Sum_of_fitness
    
    return speedlist_evaluation


In [66]:
#평가한 값 
Group_assessment_value = Score_assessment_group(Score_per_all)

#                                         정렬   점수값이 높은 순으로
sortand_Group_assessmend = sorted(Group_assessment_value, key=operator.itemgetter(0),reverse=True)
#상위 save_value 개값        
save_assessmend_value = sortand_Group_assessmend[0:save_value]
# Score_per_all에서 몇번 배열이 상위 5개인지만 
# 리스트 해쉬값만
save_assessmend_num = [save_assessmend_value[i][2] for i in range(save_value)]

print(save_assessmend_num)

print(Score_per_all[save_assessmend_num[0]])

Group_assessment_value

[11, 19, 8, 27, 7]
7103472.0


[[6890016.0, 0.03344915549865555, 0],
 [6826848.0, 0.06659164752156024, 1],
 [6682800.0, 0.09903482564573518, 2],
 [6813072.0, 0.13211043893190696, 3],
 [6787728.0, 0.1650630139846468, 4],
 [6849072.0, 0.1983133974887617, 5],
 [6825408.0, 0.23144889870294866, 6],
 [6931632.0, 0.26510008857354644, 7],
 [7021968.0, 0.29918983517770287, 8],
 [6857040.0, 0.3324789011567225, 9],
 [6862848.0, 0.3657961633975703, 10],
 [7103472.0, 0.40028158977514994, 11],
 [6861936.0, 0.43359442450380986, 12],
 [6753888.0, 0.46638271555168315, 13],
 [6791184.0, 0.49935206854534553, 14],
 [6667008.0, 0.5317185808005828, 15],
 [6824592.0, 0.5648501205564963, 16],
 [6766320.0, 0.5976987655862993, 17],
 [6908448.0, 0.6312374034365418, 18],
 [7098576.0, 0.6656990610644812, 19],
 [6893904.0, 0.6991670917466746, 20],
 [6915264.0, 0.7327388194248475, 21],
 [6928800.0, 0.7663762607049671, 22],
 [6690384.0, 0.7988562570883887, 23],
 [6906192.0, 0.8323839426716401, 24],
 [6930576.0, 0.8660300059491782, 25],
 [6918672.0

In [8]:
def roulettes(Group_assessment_value):
    roulette = random.random()
    for i in range(N):
        if i == 0:
            if 0 < roulette < Group_assessment_value[i][1]:
                break
        if Group_assessment_value[i-1][1] < roulette < Group_assessment_value[i][1]:
            break
    return i

In [68]:
Selected_list_in_roulette = []

#룰렛에 선택된 리스트들 마지막 5개는 상위5개만 
for i in range(N-5):
    Selected_list_in_roulette.append(roulettes(Group_assessment_value))

Selected_list_in_roulette += save_assessmend_num


Selected_list_in_roulette

[26,
 4,
 29,
 22,
 13,
 8,
 3,
 1,
 14,
 3,
 13,
 5,
 22,
 20,
 15,
 21,
 6,
 18,
 19,
 23,
 26,
 26,
 14,
 0,
 26,
 11,
 19,
 8,
 27,
 7]

In [69]:
len(Selected_list_in_roulette)

30

In [9]:
#부모 변경
def Change_parent(speedlist, Selected_list_in_roulette):
    W = [[[0 for j in range(Station_section)]for k in range(One_day_train)]for z in range(N)]
    # 교차함수 및 변이함수 (자식만들기)
    for i in range(int(N/2)):
        for j in range(One_day_train):
            for k in range(Station_section):
                #교차
                selection = random.random()
                ###############################################################
                if selection < Pc:
                    rand = random.choice([i+1 for i in range(listN-1)]) # 0~ 이구간, 이구간 ~ 끝 교차 , 이구간 = 1~리스트 크기 -1 
                    # W(교차된 자식) = Selected_list_in_roulette(선택된 리스트에서) rand구간으로 나눈 값
                    W[i*2][j][k]     = speedlist[Selected_list_in_roulette[i*2]][j][k][0:rand]         + speedlist[Selected_list_in_roulette[i*2+1]][j][k][rand:]
                    W[i*2+1][j][k] = speedlist[Selected_list_in_roulette[i*2+1]][j][k][0:rand] + speedlist[Selected_list_in_roulette[i*2]][j][k][rand:]
                else:
                    W[i*2][j][k]     = speedlist[Selected_list_in_roulette[i*2]][j][k]
                    W[i*2+1][j][k] = speedlist[Selected_list_in_roulette[i*2+1]][j][k]
                #변의
                for z in range(2):
                    # Pm * 100 분의 1의 확률 
                    if randint(0, 100 * Pm) == 0:
                        #selection 부분을 변의 1 -> 0  0 -> 1
                        if W[i*2+z][j][k][rand] == 0:
                            W[i*2+z][j][k][rand] = 1
                        else:
                            W[i*2+z][j][k][rand] = 0
    return W

In [71]:
test_new_lest = Change_parent(speedlist, Selected_list_in_roulette)

In [72]:
print(len(test_new_lest), len(test_new_lest[0]), len(test_new_lest[0][0]),len(test_new_lest[0][0][0]))

30 100 11 8


In [73]:
test_Score_per_all = Make_Score_per_all(test_new_lest)
test_Score_per_all

[6982512.0,
 6774096.0,
 6850704.0,
 6955296.0,
 6783840.0,
 7007664.0,
 6838224.0,
 6784512.0,
 6819696.0,
 6867408.0,
 6783600.0,
 6900672.0,
 6916560.0,
 6869952.0,
 6664176.0,
 6861360.0,
 6866016.0,
 6977568.0,
 7089120.0,
 6697344.0,
 6918672.0,
 6918672.0,
 6768432.0,
 6915264.0,
 6952416.0,
 7106016.0,
 7094160.0,
 7005360.0,
 6968160.0,
 6900720.0]

In [10]:
#세대 반복
def Evolution(speedlist, num_generation):
    while 1: 
        if num_generation >= largest_generation:
            break

        #전체 값으로 해서 
        Score_per_all = Make_Score_per_all(speedlist)

        #평가한 값 
        Group_assessment_value = Score_assessment_group(Score_per_all)

        #                                         정렬   점수값이 높은 순으로
        sortand_Group_assessmend = sorted(Group_assessment_value, key=operator.itemgetter(0),reverse=True)
        #상위 save_value 개값        
        save_assessmend_value = sortand_Group_assessmend[0:save_value]
        # Score_per_all에서 몇번 배열이 상위 5개인지만 
        # 리스트 해쉬값만
        save_assessmend_num = [save_assessmend_value[i][2] for i in range(save_value)]


        Selected_list_in_roulette = []

        #룰렛에 선택된 리스트들 마지막 5개는 상위5개만 
        for i in range(N-5):
            Selected_list_in_roulette.append(roulettes(Group_assessment_value))

        Selected_list_in_roulette += save_assessmend_num
        
        #룰렛 랜덤하게  근데이거하면 오류남 왜지?
        #Selected_list_in_roulette = random.shuffle(Selected_list_in_roulette)
        # ------------------------------------------------------------------------------------------------------------------------------------------------------------
        
        New_list = Change_parent(speedlist, Selected_list_in_roulette)
        
        New_Score_per_all = Make_Score_per_all(New_list)
        
        print("-" * 15)    
        print(num_generation,"번째를 합니다.")
        print("-" * 15)
        for i, nl in enumerate(New_Score_per_all):
            print(i, nl)
        
        speedlist = New_list
        num_generation += 1

    return New_list

In [11]:
test_list = []

test_list = Evolution(speedlist, 0)

---------------
0 번째를 합니다.
---------------
0 6878352.0
1 6793248.0
2 6821328.0
3 6917712.0
4 6884160.0
5 6749232.0
6 6791952.0
7 6862992.0
8 6799584.0
9 6965472.0
10 6877728.0
11 6756336.0
12 6835152.0
13 7063632.0
14 6660048.0
15 6811200.0
16 6864384.0
17 7015872.0
18 6880368.0
19 6997392.0
20 6781056.0
21 7103856.0
22 6850848.0
23 6830832.0
24 6777024.0
25 7047264.0
26 7048128.0
27 7014144.0
28 6946224.0
29 6972672.0
---------------
1 번째를 합니다.
---------------
0 6723936.0
1 6979824.0
2 6844992.0
3 6891600.0
4 6730560.0
5 7057968.0
6 7072704.0
7 6937488.0
8 6925968.0
9 7032960.0
10 6841056.0
11 6867072.0
12 6893952.0
13 6761952.0
14 7041936.0
15 6843408.0
16 6884256.0
17 7003584.0
18 6872352.0
19 6850656.0
20 6685200.0
21 6794208.0
22 6668448.0
23 6929424.0
24 6849456.0
25 7148832.0
26 7057344.0
27 7054608.0
28 7028640.0
29 7035360.0
---------------
2 번째를 합니다.
---------------
0 6950784.0
1 7054080.0
2 6827088.0
3 7022976.0
4 6959040.0
5 6877920.0
6 6692736.0
7 6798912.0
8 7075632.0
9 7

---------------
20 번째를 합니다.
---------------
0 7167648.0
1 7207920.0
2 7218048.0
3 7223136.0
4 7200240.0
5 7199808.0
6 7185120.0
7 7138320.0
8 7213920.0
9 7124448.0
10 7160976.0
11 7112688.0
12 7139856.0
13 7166352.0
14 7111824.0
15 7194096.0
16 7155360.0
17 7149936.0
18 7139616.0
19 7140240.0
20 7098240.0
21 7213392.0
22 7149072.0
23 7191552.0
24 7252512.0
25 7214688.0
26 7236192.0
27 7219440.0
28 7227120.0
29 7228320.0
---------------
21 번째를 합니다.
---------------
0 7230096.0
1 7225344.0
2 7207056.0
3 7151616.0
4 7119408.0
5 7208880.0
6 7207488.0
7 7128816.0
8 7194096.0
9 7225920.0
10 7237584.0
11 7225728.0
12 7129632.0
13 7187856.0
14 7241280.0
15 7097328.0
16 7165536.0
17 7168176.0
18 7203648.0
19 7157808.0
20 7239744.0
21 7122624.0
22 7193232.0
23 7190400.0
24 7159968.0
25 7226256.0
26 7236288.0
27 7228032.0
28 7221984.0
29 7229808.0
---------------
22 번째를 합니다.
---------------
0 7211232.0
1 7134768.0
2 7220880.0
3 7135824.0
4 7180176.0
5 7173792.0
6 7226880.0
7 7192992.0
8 7196448.0


---------------
40 번째를 합니다.
---------------
0 7298928.0
1 7315440.0
2 7305312.0
3 7303104.0
4 7312656.0
5 7321872.0
6 7313424.0
7 7319856.0
8 7253088.0
9 7297152.0
10 7301328.0
11 7285584.0
12 7302816.0
13 7328448.0
14 7325328.0
15 7320048.0
16 7280544.0
17 7328064.0
18 7311360.0
19 7289280.0
20 7320000.0
21 7321776.0
22 7325904.0
23 7293360.0
24 7312800.0
25 7323792.0
26 7337808.0
27 7325808.0
28 7326432.0
29 7324800.0
---------------
41 번째를 합니다.
---------------
0 7296480.0
1 7314672.0
2 7331184.0
3 7320000.0
4 7309776.0
5 7342656.0
6 7295424.0
7 7295184.0
8 7296288.0
9 7326864.0
10 7306512.0
11 7249296.0
12 7314864.0
13 7321776.0
14 7322208.0
15 7296384.0
16 7317120.0
17 7318560.0
18 7315104.0
19 7334688.0
20 7319568.0
21 7322112.0
22 7316544.0
23 7298784.0
24 7286592.0
25 7336800.0
26 7332048.0
27 7323024.0
28 7340976.0
29 7311264.0
---------------
42 번째를 합니다.
---------------
0 7295040.0
1 7316640.0
2 7331232.0
3 7333824.0
4 7270176.0
5 7291104.0
6 7313616.0
7 7294032.0
8 7289328.0


---------------
60 번째를 합니다.
---------------
0 7368720.0
1 7388352.0
2 7389408.0
3 7394544.0
4 7389120.0
5 7394256.0
6 7389024.0
7 7362336.0
8 7375680.0
9 7390368.0
10 7390752.0
11 7385952.0
12 7358496.0
13 7395744.0
14 7378800.0
15 7352592.0
16 7367520.0
17 7406064.0
18 7386240.0
19 7399536.0
20 7371984.0
21 7385232.0
22 7378368.0
23 7375920.0
24 7368240.0
25 7391088.0
26 7399296.0
27 7399440.0
28 7377600.0
29 7403184.0
---------------
61 번째를 합니다.
---------------
0 7401840.0
1 7404240.0
2 7378128.0
3 7411536.0
4 7390080.0
5 7375152.0
6 7372032.0
7 7383120.0
8 7372512.0
9 7384368.0
10 7401648.0
11 7383264.0
12 7396560.0
13 7392576.0
14 7387056.0
15 7395888.0
16 7383168.0
17 7384320.0
18 7382112.0
19 7371648.0
20 7391136.0
21 7398864.0
22 7375968.0
23 7372032.0
24 7387584.0
25 7404864.0
26 7395120.0
27 7407744.0
28 7399440.0
29 7403328.0
---------------
62 번째를 합니다.
---------------
0 7404912.0
1 7389792.0
2 7374432.0
3 7386912.0
4 7396176.0
5 7384800.0
6 7387152.0
7 7386816.0
8 7381728.0


---------------
80 번째를 합니다.
---------------
0 7454160.0
1 7477392.0
2 7461168.0
3 7445136.0
4 7441344.0
5 7448448.0
6 7464864.0
7 7462800.0
8 7444896.0
9 7452480.0
10 7467840.0
11 7439184.0
12 7460640.0
13 7463088.0
14 7461168.0
15 7449408.0
16 7451760.0
17 7468128.0
18 7456512.0
19 7451232.0
20 7440528.0
21 7451472.0
22 7446000.0
23 7461552.0
24 7458912.0
25 7482432.0
26 7473696.0
27 7465776.0
28 7457040.0
29 7470192.0
---------------
81 번째를 합니다.
---------------
0 7460928.0
1 7445184.0
2 7448496.0
3 7460112.0
4 7461696.0
5 7471248.0
6 7440576.0
7 7453056.0
8 7466496.0
9 7466784.0
10 7457616.0
11 7450416.0
12 7448976.0
13 7441008.0
14 7468272.0
15 7454832.0
16 7445760.0
17 7480128.0
18 7449072.0
19 7461312.0
20 7461024.0
21 7453824.0
22 7465296.0
23 7463952.0
24 7474464.0
25 7471968.0
26 7476240.0
27 7473648.0
28 7468848.0
29 7469760.0
---------------
82 번째를 합니다.
---------------
0 7463904.0
1 7443168.0
2 7459584.0
3 7453776.0
4 7460208.0
5 7454736.0
6 7474752.0
7 7470048.0
8 7441344.0


...

---------------
320 번째를 합니다.
---------------
0 7884096.0
1 7882032.0
2 7871904.0
3 7866048.0
4 7870320.0
5 7874112.0
6 7869120.0
7 7868448.0
8 7877184.0
9 7888512.0
10 7894320.0
11 7864080.0
12 7881408.0
13 7882368.0
14 7876560.0
15 7891152.0
16 7889568.0
17 7894320.0
18 7894368.0
19 7890096.0
20 7873536.0
21 7882368.0
22 7891440.0
23 7888416.0
24 7872192.0
25 7899264.0
26 7899552.0
27 7887648.0
28 7892208.0
29 7893408.0
---------------
321 번째를 합니다.
---------------
0 7874448.0
1 7894032.0
2 7879152.0
3 7875504.0
4 7873584.0
5 7883856.0
6 7875216.0
7 7876512.0
8 7892832.0
9 7885968.0
10 7888224.0
11 7886688.0
12 7898304.0
13 7887792.0
14 7869648.0
15 7887216.0
16 7884384.0
17 7894320.0
18 7892688.0
19 7889232.0
20 7888896.0
21 7870656.0
22 7897536.0
23 7869936.0
24 7879920.0
25 7891584.0
26 7899648.0
27 7895472.0
28 7890960.0
29 7899984.0
---------------
322 번째를 합니다.
---------------
0 7899360.0
1 7880064.0
2 7888032.0
3 7890672.0
4 7866192.0
5 7886688.0
6 7886544.0
7 7884960.0
8 7889232

---------------
340 번째를 합니다.
---------------
0 7939440.0
1 7931856.0
2 7947840.0
3 7923888.0
4 7931088.0
5 7931712.0
6 7924320.0
7 7939584.0
8 7938864.0
9 7936848.0
10 7936176.0
11 7935936.0
12 7929024.0
13 7928640.0
14 7932288.0
15 7936944.0
16 7939200.0
17 7940784.0
18 7924512.0
19 7933728.0
20 7920912.0
21 7936608.0
22 7931760.0
23 7926336.0
24 7944720.0
25 7941744.0
26 7936224.0
27 7946784.0
28 7944288.0
29 7939536.0
---------------
341 번째를 합니다.
---------------
0 7938816.0
1 7944000.0
2 7942704.0
3 7940544.0
4 7931952.0
5 7933680.0
6 7935504.0
7 7931856.0
8 7935600.0
9 7939008.0
10 7946256.0
11 7943232.0
12 7928112.0
13 7939968.0
14 7929168.0
15 7935936.0
16 7916592.0
17 7931664.0
18 7935840.0
19 7928544.0
20 7936464.0
21 7924608.0
22 7916352.0
23 7927440.0
24 7937904.0
25 7946064.0
26 7944096.0
27 7947984.0
28 7941744.0
29 7941984.0
---------------
342 번째를 합니다.
---------------
0 7931952.0
1 7931952.0
2 7940112.0
3 7940112.0
4 7952880.0
5 7932720.0
6 7933008.0
7 7940736.0
8 7937376

---------------
360 번째를 합니다.
---------------
0 7949904.0
1 7957872.0
2 7953888.0
3 7965504.0
4 7953840.0
5 7955040.0
6 7954032.0
7 7959984.0
8 7954080.0
9 7964160.0
10 7953840.0
11 7953504.0
12 7961184.0
13 7950000.0
14 7945392.0
15 7955376.0
16 7949568.0
17 7956096.0
18 7956240.0
19 7942800.0
20 7950096.0
21 7961424.0
22 7960896.0
23 7945296.0
24 7963920.0
25 7969584.0
26 7968816.0
27 7968048.0
28 7959168.0
29 7966752.0
---------------
361 번째를 합니다.
---------------
0 7956624.0
1 7956240.0
2 7963104.0
3 7962336.0
4 7958016.0
5 7958736.0
6 7959408.0
7 7948848.0
8 7965024.0
9 7969008.0
10 7962192.0
11 7948416.0
12 7962672.0
13 7955040.0
14 7952688.0
15 7963776.0
16 7957584.0
17 7953600.0
18 7955760.0
19 7952976.0
20 7960512.0
21 7961520.0
22 7966800.0
23 7953792.0
24 7966656.0
25 7962192.0
26 7970256.0
27 7966608.0
28 7967904.0
29 7962720.0
---------------
362 번째를 합니다.
---------------
0 7959504.0
1 7955232.0
2 7960512.0
3 7964304.0
4 7970544.0
5 7965696.0
6 7964976.0
7 7962576.0
8 7970640

---------------
380 번째를 합니다.
---------------
0 7981968.0
1 7986912.0
2 7984704.0
3 7981920.0
4 7991280.0
5 7978704.0
6 7987824.0
7 7985952.0
8 7980528.0
9 7982112.0
10 7976160.0
11 7987440.0
12 7992720.0
13 7986816.0
14 7991040.0
15 7985088.0
16 7986960.0
17 7987728.0
18 7986048.0
19 7985280.0
20 7976544.0
21 7985664.0
22 7989504.0
23 7989456.0
24 7993776.0
25 7995168.0
26 7990368.0
27 7986432.0
28 7989888.0
29 7991136.0
---------------
381 번째를 합니다.
---------------
0 7985664.0
1 7984080.0
2 7987632.0
3 7981392.0
4 7978416.0
5 7983360.0
6 7985184.0
7 7988016.0
8 7979808.0
9 7989408.0
10 7984704.0
11 7976592.0
12 7993392.0
13 7988400.0
14 7990992.0
15 7987776.0
16 7991472.0
17 7988160.0
18 7987872.0
19 7983024.0
20 7988928.0
21 7989696.0
22 7988592.0
23 7988544.0
24 7982976.0
25 7988784.0
26 7994112.0
27 7992480.0
28 7991664.0
29 7993872.0
---------------
382 번째를 합니다.
---------------
0 7985424.0
1 7983312.0
2 7989984.0
3 7991520.0
4 7991664.0
5 7991280.0
6 7988928.0
7 7988448.0
8 7986192

KeyboardInterrupt: 

In [12]:
test_list

[]

In [51]:
if speedlist[1][1][1][1] == 0:
    print("!!")
else:
    print("??")

??


In [ ]:
test_list2 = []

test_list2 = Evolution(test_list, 0)

In [37]:
Www = [[[0 for i in range(2)]for j in range(2)]for k in range(2)]
    # 교차함수 및 변이함수 (자식만들기)
    
print(Www)
for i in range(2):
    for j in range(2):
        for k in range(2):
            Www[i][j][k] = [43,43,0,2,2]
            
Www

[[[0, 0], [0, 0]], [[0, 0], [0, 0]]]


[[[[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]],
  [[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]]],
 [[[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]],
  [[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]]]]